<a href="https://colab.research.google.com/github/Puli-vigneswar/BIZCARDX_PROJECT/blob/main/bizcard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTING PACKAGES
import streamlit as st
import easyocr
from PIL import Image
import pandas as pd
import numpy as np
import re
import cv2
import mysql.connector
import matplotlib.pyplot as plt

st.title(":rainbow[BUSINESS CARD DATA EXTRACTION] ")
#connecting to sql database
mydb = mysql.connector.connect(host="localhost",
                             user="root",
                             password="xxxxxxxxx",
                             database="BUSINESS_CARDS")

cursor = mydb.cursor()
CREATE_QUERY='''CREATE TABLE IF NOT EXISTS CARDS_DATA(
                                                        EMPLOYEE_NAME VARCHAR(50),
                                                        COMPANY_NAME VARCHAR(50),
                                                        DESIGNATION VARCHAR(50),
                                                        CONTACT VARCHAR(50),
                                                        URL VARCHAR(50),
                                                        EMAIL VARCHAR(50),
                                                        ADDRESS TEXT,
                                                        PINCODE VARCHAR(50)
                                                        )
                                                        '''
cursor.execute(CREATE_QUERY)
mydb.commit()
#initialising dictionary
data={"employee_name":str(),
      "company_name":str(),
      "Designation":str(),
      "contact":str(),
      "URL":str(),
      "email":str(),
      "address":str(),
      "pin_code":str()
      }
# Initialize the EasyOCR reader
reader = easyocr.Reader(['en'])
#deploying streamlit tabs
HOME,UPLOAD,EDIT,VIEW=st.tabs(["HOME","UPLOAD","EDIT","VIEW"])

with HOME:
    st.balloons()
    st.write(":rainbow[<----------------------------------------------------------------------------------------------------->]")
    st.subheader(":orange[Welcome To the Data Extraction Model BIZCARDX ]")
    st.markdown(":violet[This is an applicaion To retrive the text]")
    st.markdown(":violet[From the image of business card]")
    st.markdown(":violet[Upload The Data In an Efficient Manner with Assistance of this Application]")
    st.markdown(" ")
    st.subheader(":blue[Technologies Used:]")
    st.markdown(":orange[***EASY OCR(OPTICAL CHARACTER READING)***]")
    st.markdown(":red[***Database- MYSQL***]")
    st.markdown(":red[***Python as base scripting language***]")

# File uploader
with UPLOAD:
    st.write(":rainbow[<----------------------------------------------------------------------------------------------->]")
    st.header("upload the image to be processed")
    uploaded_file = st.file_uploader(":orange[Upload your card image here]", type=['png', 'jpg', 'jpeg'])
    if uploaded_file is not None:
    # Convert the file to an image
        image = Image.open(uploaded_file)

    # Display the uploaded image
        st.image(image, caption='Uploaded Image', use_column_width=True)

    # Convert the image to a NumPy array
        img_array = np.array(image)

    # Use EasyOCR to extract text
        result = reader.readtext(img_array)
        result2=reader.readtext(img_array,paragraph=True)
    # Display the extracted text

        n=[]
        for i in result:
            n.append(i[1])

        nd2=[]
        for i in result2:
            nd2.append(i[1])

            for i, string in enumerate(n):
                    if i == 0:
                        data["employee_name"]=string

                    elif i == 1:
                        data["Designation"]=string

                    #mobilenumber
            for i, string in enumerate(n):
                    match = re.search(r'(?:ph|phone|phno)?(?:[+-]?\d*){7,}', string)
                    match = re.search(r'(?:ph|phone|phno)?\s*(?:[+-]?\d\s*){7,}', string)
                    match = re.search(r'(?:ph|phone|phno)?\s*(?:[+-]?\d\s*[\(\)]*){7,}', string)
                    if match and len(re.findall(r'\d', string)) > 7:
                            data['contact']=string
                # TO FIND EMAIL
            eid =''
            match = re.search(r'\d{6,7}', string.lower())
            for i, string in enumerate(n):
                        if re.search(r'@', string.lower()):
                            EMAIL=string.lower()
                            eid=string
            data["email"]=eid

                    #address
            for i, string in enumerate(n):
                            match = re.search(r'\d{6,7}', string.lower())
                            if match:
                                PIN=match.group()
                                adr=string
            data["pin_code"]=PIN
            data["address"]=adr

                # To get WEBSITE_URL
            web=''
            for i,string in enumerate(nd2):
                    if re.search(r'www', string.lower()):
                        web=string.lower()
            data["URL"]=web

                    #companyname
            for item,string in enumerate(nd2):
                if item == len(nd2)-1:
                    data["company_name"]=string
        if st.button("SHOW EXTRACTED DATA"):
            st.dataframe(data.items())

    st.subheader("IF you would like to make changes in the data")
    st.subheader('Switch to the :orange[EDIT] Tab')




with EDIT:

        st.write(":grey[<------------------------------------------------------------------------------------------------------------>]")
        data = {k: [v] for k, v in data.items()}
    # Function to update the dictionary
        def update_dict(key, new_value):
                data[key] = new_value

    # Select a key to update
        selected_key = st.selectbox('Select a key to update', list(data.keys()))


        for i in data:
            if i==selected_key:
                val=str(data[i])
                data[i]=st.text_input("Enter New Value",val)
        st.warning("kindly Refer to the Given Formats to update values")


        options=["TEXT ","Paragraph","Image"]
        opted=st.selectbox('Choose the required format to edit data ',options)

        if opted=='text':
                for i in n:
                    st.write(i)
        if opted=='paragraph':
                for i in nd2:
                    st.write(i)

        if opted=='image':
                st.image(image, caption='Uploaded Image', use_column_width=True)

        if st.button(":violet[SHOW the Updated Data] "):
            st.dataframe(data)

        df=pd.DataFrame(data)

        if st.button(":green[Upload Data to Database]"):
            for index,row in df.iterrows():
                                    insert_query='''INSERT INTO CARDS_DATA(EMPLOYEE_NAME ,
                                                                    COMPANY_NAME,
                                                                    DESIGNATION,
                                                                    CONTACT,
                                                                    URL,
                                                                    EMAIL,
                                                                    ADDRESS,
                                                                    PINCODE)
                                                                                    values(%s,%s,%s,%s,%s,%s,%s,%s)'''
                                    values=(row["employee_name"],
                                            row["company_name"],
                                            row['Designation'],
                                            row['contact'],
                                            row['URL'],
                                            row['email'],
                                            row['address'],
                                            row['pin_code'])
                                    try:
                                            cursor.execute(insert_query,values)
                                            mydb.commit()
                                    except:
                                            print("channel values already inserted")
with VIEW:
    st.write("<------------------------------------------------------------------------------------------------------------------------>")
    st.header("RETRIEVE THE DATABASE ")

    if st.button(":green[View Database]"):
            cursor.execute("select * from CARDS_DATA ")
            q1=cursor.fetchall()
            df1=pd.DataFrame(q1)
            st.dataframe(df1)

    if st.button(":red[DELETE DATA]"):
            cursor.execute("TRUNCATE TABLE CARDS_DATA")













